In [1]:
%load_ext autoreload
%autoreload 2

# Import necessary modules 
import numpy as np
import pandas as pd
from itertools import product
import plotly.graph_objs as go
import plotly.express as px
from numpy.linalg import eigvalsh
from plotly.subplots import make_subplots
import torch
import itertools
import concurrent.futures
from tqdm import tqdm

from rbf_volatility_surface import RBFVolatilitySurface
from smoothness_prior import RBFQuadraticSmoothnessPrior
from dataset_sabr import generate_sabr_call_options
from surface_vae_trainer import SurfaceVAETrainer

In [2]:
# Define the strike price list and maturity time list
strike_price_list = np.array([0.75, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2, 1.3, 1.5])
maturity_time_list = np.array([0.02, 0.08, 0.17, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0])

# Create the product grid of maturity times and strike prices
product_grid = list(product(maturity_time_list, strike_price_list))
maturity_times, strike_prices = zip(*product_grid)

# Convert to arrays for further operations
maturity_times = np.array(maturity_times)
strike_prices = np.array(strike_prices)

# Variance formula for log-uniform distribution
def log_uniform_variance(a, b):
    log_term = np.log(b / a)
    var = ((b ** 2 - a ** 2) / (2 * log_term)) - ((b - a) / log_term) ** 2
    return var

# Calculate standard deviations for maturity times and strike prices
maturity_std = np.sqrt(log_uniform_variance(maturity_time_list.min(), maturity_time_list.max()))
strike_std = np.sqrt(log_uniform_variance(strike_price_list.min(), strike_price_list.max()))

# Define the SABR model parameters
alpha = 0.20  # Stochastic volatility parameter
beta = 0.50   # Elasticity parameter
rho = -0.75   # Correlation between asset price and volatility
nu = 1.0      # Volatility of volatility parameter

# Other model parameters
risk_free_rate = np.log(1.02)  # Risk-free interest rate
underlying_price = 1.0         # Current price of the underlying asset

# Generate the dataset using the SABR model and Black-Scholes formula
call_option_dataset = generate_sabr_call_options(
    alpha=alpha,
    beta=beta,
    rho=rho,
    nu=nu,
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    risk_free_rate=risk_free_rate,
    underlying_price=underlying_price
)

# Maturity times and strike prices from the previous product grid setup
hypothetical_maturity_time_list = np.logspace(np.log10(0.01), np.log10(3.1), 100)
hypothetical_strike_price_list = np.logspace(np.log10(0.7), np.log10(1.75), 100)

# Create the product grid of maturity times and strike prices
hypothetical_product_grid = list(product(hypothetical_maturity_time_list, hypothetical_strike_price_list))
hypothetical_maturity_times, hypothetical_strike_prices = zip(*hypothetical_product_grid)
hypothetical_maturity_times, hypothetical_strike_prices = np.array(hypothetical_maturity_times), np.array(hypothetical_strike_prices)

# Reshape the data for 3D surface plotting
hypothetical_maturities_grid = hypothetical_maturity_times.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))  
hypothetical_strikes_grid = hypothetical_strike_prices.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))

In [3]:
n_roots = 350
# n_roots = 10
smoothness_controller = 3.274549162877732e-05

# Initialize the RBFQuadraticSmoothnessPrior class
smoothness_prior = RBFQuadraticSmoothnessPrior(
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    maturity_std=maturity_std,
    strike_std=strike_std,
    n_roots=n_roots,
    smoothness_controller=smoothness_controller,
    random_state=0,
)

prior_covariance_matrix = smoothness_prior.prior_covariance()
prior_eigenvalues = np.sort(np.linalg.eigvalsh(prior_covariance_matrix))[::-1].copy()

# The constant_volatility is set to a reasonable value
constant_volatility = RBFVolatilitySurface.calculate_constant_volatility(
    call_option_dataset["Implied Volatility"],
    call_option_dataset["Time to Maturity"],
    call_option_dataset["Strike Price"],
    risk_free_rate,
    underlying_price
)

sampled_surface_coefficients = smoothness_prior.sample_smooth_surfaces(10000)

In [4]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [ ]:
latent_dim = 70  # Latent dimension
data_dim = 100  # Data dimension of input
latent_diagonal = prior_eigenvalues[:latent_dim]  # Eigenvalues for latent prior
batch_size = 1000  # Batch size for training
beta_ = 1.0  # Beta value for beta-VAE
fine_tune_learning_rate = 1e-4  # Fine-tune learning rate
pre_train_epochs = 350  # Number of pre-train epochs
fine_tune_epochs = 20  # Number of fine-tune epochs
device = "cpu"  # Use CPU as the device

# Define the hyperparameter grid
hidden_dim_grid = [128, 256, 512]  # Example grid for hidden_dim
n_layers_grid = [2, 4, 8]         # Example grid for n_layers
pre_train_learning_rate_grid = [1e-4, 1e-3, 1e-2]  # Example grid for learning rate

# Initialize an empty DataFrame to store the results
results_df = pd.DataFrame()

# Define the grid search
grid = itertools.product(hidden_dim_grid, n_layers_grid, pre_train_learning_rate_grid)

for hidden_dim, n_layers, pre_train_learning_rate in tqdm(grid):
    # Initialize the trainer with the specified configuration
    trainer = SurfaceVAETrainer(
        latent_dim=latent_dim,
        hidden_dim=hidden_dim,
        n_layers=n_layers,
        data_dim=data_dim,
        latent_diagonal=latent_diagonal,
        batch_size=batch_size,
        beta=beta_,
        pre_train_learning_rate=pre_train_learning_rate,
        fine_tune_learning_rate=fine_tune_learning_rate,
        pre_train_epochs=pre_train_epochs,
        fine_tune_epochs=fine_tune_epochs,
        device=device,
    )

    # Train the model using pre_train
    trainer.pre_train_with_sampling(
        smoothness_prior=smoothness_prior,
        experiment_name=f"test_hd_{hidden_dim}_nl_{n_layers}_lr_{pre_train_learning_rate}"
    )

    # Retrieve the last row of the loss history (assuming it's stored in trainer.pre_train_loss_history)
    loss_df = pd.DataFrame(trainer.pre_train_loss_history)
    last_row = loss_df.iloc[-1].copy()

    # Add the configuration as columns in the last row
    last_row['hidden_dim'] = hidden_dim
    last_row['n_layers'] = n_layers
    last_row['pre_train_learning_rate'] = pre_train_learning_rate

    results_df = pd.concat([results_df, pd.DataFrame([last_row])], ignore_index=True)

In [14]:
# Rank the losses for each column (except 'Total Loss')
ranked_losses = results_df.drop(columns=['Total Loss', 'hidden_dim', 'n_layers', 'pre_train_learning_rate']).rank()

ranked_df = results_df.copy()

# Compute the average rank for each configuration
ranked_df['average_rank'] = ranked_losses.mean(axis=1)

# Sort by the average rank (lower is better)
ranked_df = ranked_df.sort_values('average_rank')

# Print the top-ranked configurations
ranked_df

,Reconstruction Loss,KL Loss,Total Loss,hidden_dim,n_layers,pre_train_learning_rate,average_rank
24,0.050729,2.842956,2.893685,512.0,8.0,0.0001,5.5
16,0.051739,0.690140,0.741879,256.0,8.0,0.0010,7.0
13,0.051627,2.068588,2.120214,256.0,4.0,0.0010,7.0
25,0.051970,0.259207,0.311176,512.0,8.0,0.0010,8.0
21,0.050912,9.276610,9.327522,512.0,4.0,0.0001,8.5
10,0.051512,3.765398,3.816909,256.0,2.0,0.0010,8.5
7,0.051636,3.176954,3.228590,128.0,8.0,0.0010,9.0
19,0.051809,3.002637,3.054446,512.0,2.0,0.0010,9.5
17,0.051310,12.442724,12.494034,256.0,8.0,0.0100,9.5
22,0.052277,0.581075,0.633352,512.0,4.0,0.0010,10.0


In [31]:
latent_dim = 70  # Latent dimension
data_dim = 100  # Data dimension of input
hidden_dim = 512
n_layers = 8
latent_diagonal = prior_eigenvalues[:latent_dim]  # Eigenvalues for latent prior
batch_size = 1000  # Batch size for training
beta_ = 1.0  # Beta value for beta-VAE
pre_train_learning_rate = 1e-3
fine_tune_learning_rate = 1e-4  # Fine-tune learning rate
pre_train_epochs = 600  # Number of pre-train epochs
fine_tune_epochs = 20  # Number of fine-tune epochs
device = "cpu"  # Use CPU as the device

trainer = SurfaceVAETrainer(
    latent_dim=latent_dim,
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    data_dim=data_dim,
    latent_diagonal=latent_diagonal,
    batch_size=batch_size,
    beta=beta_,
    pre_train_learning_rate=pre_train_learning_rate,
    fine_tune_learning_rate=fine_tune_learning_rate,
    pre_train_epochs=pre_train_epochs,
    fine_tune_epochs=fine_tune_epochs,
    device=device,
)

# Train the model using pre_train
trainer.pre_train_with_sampling(
    smoothness_prior=smoothness_prior,
    experiment_name="test vae"
)

Epoch 1/600, Batch 1, Losses: {'Reconstruction Loss': 6.999632358551025, 'KL Loss': 62261.0, 'Total Loss': 62268.0}
Epoch 2/600, Batch 1, Losses: {'Reconstruction Loss': 3.442704200744629, 'KL Loss': 60993.34765625, 'Total Loss': 60996.7890625}
Epoch 3/600, Batch 1, Losses: {'Reconstruction Loss': 1.7724372148513794, 'KL Loss': 52939.8046875, 'Total Loss': 52941.578125}
Epoch 4/600, Batch 1, Losses: {'Reconstruction Loss': 0.9979664087295532, 'KL Loss': 34494.4453125, 'Total Loss': 34495.44140625}
Epoch 5/600, Batch 1, Losses: {'Reconstruction Loss': 0.47238582372665405, 'KL Loss': 13564.9208984375, 'Total Loss': 13565.3935546875}
Epoch 6/600, Batch 1, Losses: {'Reconstruction Loss': 0.5227526426315308, 'KL Loss': 12545.759765625, 'Total Loss': 12546.2822265625}
Epoch 7/600, Batch 1, Losses: {'Reconstruction Loss': 1.4964783191680908, 'KL Loss': 12437.5712890625, 'Total Loss': 12439.0673828125}
Epoch 8/600, Batch 1, Losses: {'Reconstruction Loss': 1.5469316244125366, 'KL Loss': 9895.76

In [32]:
loss_history = pd.DataFrame(trainer.pre_train_loss_history)

# Create a subplot figure with 1x2 grid for individual losses, and a second row spanning the entire width for total loss
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Reconstruction Loss", "KL Loss", "Total Loss"),
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
           [{'colspan': 2, 'type': 'scatter'}, None]],
    vertical_spacing=0.1,
    horizontal_spacing=0.1
)

# Add traces for individual losses
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["Reconstruction Loss"], mode="lines", name="Reconstruction Loss"), row=1, col=1)
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["KL Loss"], mode="lines", name="KL Loss"), row=1, col=2)

# Add a trace for the total loss spanning the entire second row
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["Total Loss"], mode="lines", name="Total Loss"), row=2, col=1)

# Update the layout to include 'Iterations' as the x-axis name for each subplot
fig.update_xaxes(title_text="Iterations", row=1, col=1)
fig.update_xaxes(title_text="Iterations", row=1, col=2)
fig.update_xaxes(title_text="Iterations", row=2, col=1)  # The third row spans two columns

fig.update_yaxes(type="log", row=1, col=1)
fig.update_yaxes(type="log", row=1, col=2)
fig.update_yaxes(type="log", row=2, col=1)  # The third row spans two columns

# Update the layout
fig.update_layout(height=900, width=900, title_text="Beta-VAE Training Losses", showlegend=False)

# Show the plot
fig.show()